In [1]:
import json
import pandas as pd

In [2]:
# dfs = [
#     pd.read_csv("Batch_4042437_batch_results.csv"),
#     pd.read_csv("Batch_4043980_batch_results.csv")]

dfs = [pd.read_csv("Batch_4060567_batch_results.csv")]

df = pd.concat(dfs)

In [15]:
df.shape

(105, 329)

In [3]:
no_answer = "The caption does not contain an answer to the question"

In [4]:
tmp = df[[c for c in df.columns if 'Item_10' in c]]

In [5]:
def convert_responses(row):
    results = {}
    s = row['Answer.taskAnswers']
    data = json.loads(s)[0]    
    for q, vals in data.items():
        if 'question' in q:
            for a, v in vals.items():
                if v == True:
                    q = q.replace("question", "").replace("answer", "")
                    results[q] = a
    return results

def process_row(row):
    data = {}
    correct = {}
    responses = convert_responses(row)
    for k, v in row.items():
        if '_Condition' in k:
            k = k.replace('Input.Item_', '').replace('_Condition', '')
            data[k] = v
        if 'True_Answers_Indices' in k:
            k = k.replace('Input.Item_', '').replace('_True_Answers_Indices', '')
            correct[k] = tuple([int(i) for i in v.split(",")])
    output = []
    for k, v in data.items():
        d = {"Condition": v, 
             "Response": responses[k], 
             "ResponseIndex": int(k), 
             'CorrectAnswers': correct[k]}
        output.append(d)    
    return output

def process_all(df):
    results = []
    for _, row in df.iterrows():
        results += process_row(row)    
    return pd.DataFrame(results)

In [6]:
results = process_all(df)

In [7]:
results['CorrectResponse'] = results.apply(
    lambda x: x['ResponseIndex'] in x['CorrectAnswers'], axis=1)

In [8]:
results.head()

,Condition,Response,ResponseIndex,CorrectAnswers,CorrectResponse
0,S0_caption,The caption does not contain an answer to the ...,1,"(10,)",False
1,S1_caption,grey,2,"(6, 7, 11)",False
2,S1-Q_caption,The caption does not contain an answer to the ...,3,"(13,)",False
3,S1-QH_caption,The caption does not contain an answer to the ...,4,"(1,)",False
4,Human_Caption,The caption does not contain an answer to the ...,5,"(11,)",False


In [9]:
results['conveys_answer'] = results['Response'] != no_answer

In [10]:
xtab = pd.crosstab(results['Condition'], results['conveys_answer'])

In [11]:
per = (xtab.T / xtab.T.sum()).T

In [16]:
per[True].sort_values().round(5)

Condition
S0_caption       0.20879
S1_caption       0.24542
Human_Caption    0.33333
S1-QH_caption    0.42125
S1-Q_caption     0.43956
Name: True, dtype: float64

In [13]:
xtab = pd.crosstab(results['Condition'], results['CorrectResponse'])

In [14]:
xtab

CorrectResponse,False,True
Condition,,
Human_Caption,252,21
S0_caption,252,21
S1-QH_caption,252,21
S1-Q_caption,252,21
S1_caption,252,21


In [ ]:
results.shape